# First: process data for Sobolev Alignment
In this tutorial, we give an example on how to process two datasets prior to use Sobolev Alignment.
<br/>
We give the example of [Kinker et al 2020] (cell lines) and [Kim et al 2020] (tumors) datasets used in the manuscript presenting Sobolev Alignment.

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

# Make a data folder
! mkdir -p ./data/

## Kim et al
### Download

In [ ]:
# Download data
! mkdir -p data
! wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE131907&format=file&file=GSE131907%5FLung%5FCancer%5Fraw%5FUMI%5Fmatrix%2Etxt%2Egz" --output-document=./data/kim.txt.gz
! wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE131907&format=file&file=GSE131907%5FLung%5FCancer%5Fcell%5Fannotation%2Etxt%2Egz" --output-document=./data/kim.cell_types.txt


### Read data and restrict to epithelial cells

In [ ]:
# Restrict to cancer cells
kim_metadata_df = pd.read_csv('./data/kim.cell_types.txt.gz', compression='gzip', sep='\t', index_col=0)
kim_epithelial_cells = kim_metadata_df.loc[kim_metadata_df['Cell_type'] == 'Epithelial cells'].index.astype(str)

In [ ]:
# Read only cancer cells
kim_df = pd.read_csv(
    './data/kim.txt.gz', 
    compression='gzip', sep='\t', 
    usecols = ['Index'] + list(kim_epithelial_cells)
)
kim_df = kim_df.set_index('Index').T

# Format as AnnData
kim_an = sc.AnnData(
    kim_df.values,
    var=pd.DataFrame(index=kim_df.columns),
    obs=kim_metadata_df.loc[kim_df.index]
)

### QC and filtering

In [ ]:
sc.pp.filter_cells(kim_an, min_genes=200)
sc.pp.filter_genes(kim_an, min_cells=3)

# # QC filtering
kim_an.var['mt'] = kim_an.var_names.str.startswith('MT')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(kim_an, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
kim_an = kim_an[kim_an.obs.n_genes_by_counts < 7500, :]
kim_an = kim_an[kim_an.obs.pct_counts_mt < 20, :]

In [ ]:
# Gene filtering
malat1 = kim_an.var_names.str.startswith('MALAT1')
mito_genes = kim_an.var_names.str.startswith('MT-')
hb_genes = kim_an.var_names.str.contains('^HB[^(P)]')

remove = np.add(mito_genes, malat1)
remove = np.add(remove, hb_genes)
keep = np.invert(remove)

kim_an = kim_an[:,keep]

### Highly variable genes

In [ ]:
sc.pp.highly_variable_genes(kim_an, flavor='seurat_v3', n_top_genes=2000)
kim_an = kim_an[:,kim_an.var.highly_variable]

# For scVI
kim_an.layers['counts'] = kim_an.X

### Save

In [ ]:
kim_an.write_h5ad('./data/kim.h5ad')

## Kinker et al
### Download
Please download the data from the Broad Institute website (https://singlecell.broadinstitute.org/single_cell/study/SCP542/pan-cancer-cell-line-heterogeneity#/). Put the UMI count files as ./data/kinker.txt and the cell typing file as './data/kinker.cell_types.txt'.

### Read data and select lung cancer cell lines

In [ ]:
# Restrict to lung cancer cells
kinker_metadata_df = pd.read_csv('./data/kinker.cell_types.txt', sep='\t', header=[0,1])
kinker_metadata_df.columns = kinker_metadata_df.columns.get_level_values(0)
kinker_lung_cancer_cell = kinker_metadata_df.loc[kinker_metadata_df['Cancer_type'] == 'Lung Cancer']['NAME'].values

In [ ]:
# Read only cancer cells
kinker_df = pd.read_csv(
    './data/kinker.txt', 
    sep='\t',
    header=[0,1,2], 
    index_col=0,
)
kinker_df = kinker_df[kinker_lung_cancer_cell]

In [ ]:
# Format as AnnData
kinker_an = sc.AnnData(
    kinker_df.values,
    var=pd.DataFrame(index=kinker_df.columns),
    obs=kinker_metadata_df.set_index('NAME').loc[kinker_df.index.get_level_values(0)]
)

### Filter and QC

In [ ]:
sc.pp.filter_cells(kinker_an, min_genes=200)
sc.pp.filter_genes(kinker_an, min_cells=3)

# # QC filtering
kinker_an.var['mt'] = kinker_an.var_names.str.startswith('MT')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(kinker_an, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
kinker_an = kinker_an[kinker_an.obs.n_genes_by_counts < 7500, :]
kinker_an = kinker_an[kinker_an.obs.pct_counts_mt < 20, :]

In [ ]:
# Gene filtering
malat1 = kinker_an.var_names.str.startswith('MALAT1')
mito_genes = kinker_an.var_names.str.startswith('MT-')
hb_genes = kinker_an.var_names.str.contains('^HB[^(P)]')

remove = np.add(mito_genes, malat1)
remove = np.add(remove, hb_genes)
keep = np.invert(remove)

kinker_an = kinker_an[:,keep]

### Highly variable genes

In [ ]:
sc.pp.highly_variable_genes(kinker_an, flavor='seurat_v3', n_top_genes=2000)
kinker_an = kinker_an[:,kinker_an.var.highly_variable]

# For scVI
kinker_an.layers['counts'] = kinker_an.X

### Save

In [ ]:
kinker_an.obs = kinker_an.obs.dropna(axis=1)
kinker_an.obs['Pool_ID'] = kinker_an.obs['Pool_ID'].astype(str)
kinker_an.write_h5ad('./data/kinker.h5ad')